# Importing Data and Dependencies

In [1]:
import numpy as np
import pandas as pd
from fastai.vision.all import *

!pip install -Uqq fastkaggle --no-index --find-links=file:///kaggle/input/library-fastkaggle/
from fastkaggle import *

from shutil import copyfile
copyfile(src = "../input/isic-vision/isic_helper.py", dst = "../working/isic_helper.py")
from isic_helper import *

In [2]:
path = setup_comp('isic-2024-challenge')
path.ls()

(#6) [Path('../input/isic-2024-challenge/sample_submission.csv'),Path('../input/isic-2024-challenge/train-metadata.csv'),Path('../input/isic-2024-challenge/test-metadata.csv'),Path('../input/isic-2024-challenge/test-image.hdf5'),Path('../input/isic-2024-challenge/train-image'),Path('../input/isic-2024-challenge/train-image.hdf5')]

In [3]:
df = pd.read_csv(path/'train-metadata.csv', low_memory=False)
test_df = pd.read_csv(path/'test-metadata.csv', low_memory=False)

In [4]:
img_path = path/'train-image.hdf5'
test_img_path = path/'test-image.hdf5'

imgs_data = get_img_data(img_path, test_img_path, df)

Reading test files: 100%|██████████| 3/3 [00:00<00:00, 179.37it/s]


# Submission

In [5]:
def get_items(path): return df.isic_id

def get_x(key): return PILImage.create(imgs_data[key][0])

def get_y(key): return imgs_data[key][1]

In [6]:
def loss_func(targs, preds): return flat_BCE_with_logits(targs, preds, ratio=3/2)

In [7]:
learn = load_learner("../input/isic-vision/fitted_resnet18.pkl", cpu=False)

In [8]:
test_keys = list(test_df.isic_id)
test_dl = learn.dls.test_dl(test_keys, with_labels=False)

In [9]:
test_preds = learn.tta(dl=test_dl)[0]

submission = pd.DataFrame({"isic_id":test_keys, "target":test_preds.numpy().reshape(-1)})
submission.to_csv("submission.csv", index=False)

In [10]:
!head submission.csv

isic_id,target
ISIC_0015657,-2.4701843
ISIC_0015729,-5.6064453
ISIC_0015740,-3.5679932
